Dataset credits: https://www.kaggle.com/arjuntejaswi/plant-village

- Potato -> Healthy, Early Blight() and Late Blight(Microogranism)

Data
Data Cleaning and Preprocessing -> tf Dataset, Data Augmentation(More training samples)
Model Building -> CNN
Export Trained model to disk
MLOPs -> tfserving API use

Quantization -> Tf Model -> Tf lite model
Deploy to Google Cloud Fucntions. <--- Application use this deployed

Import data into tensorflow dataset object

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [ ]:
import zipfile
import os

with zipfile.ZipFile("/content/PlantVillage-PotatoDisease.zip", 'r') as zip_ref:
    zip_ref.extractall("PotatoDatasett")

In [ ]:
! rm -rf "/content/PotatoDatasett/__MACOSX"

We will use image_dataset_from_directory api to load all images in tensorflow dataset:
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory

In [ ]:
IMAGE_SIZE=256 #use by tf dataset
BATCH_SIZE=32
RGB_CHANNELS=3
EPOCHS=30

In [ ]:
# Directly using tf dataset for all images
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/PotatoDatasett/PlantVillage-PotatoDisease",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
# return a tf.data.Dataset with len = number of batches
# tf dataset already deals with batches
# .filer .map

In [ ]:
class_names = dataset.class_names
class_names
# Basically folder names

In [ ]:
print("total number of batches",len(dataset))
# total batches if len*32 approx size of all data

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    # print(image_batch[0])
    # visualize using plt
    # plt.imshow(image_batch[0].numpy().astype("uint8"))
    # plt.title(labels_batch[0].numpy())
    # classnames[label[index]]
    print(labels_batch.numpy())

# Each batch 32 images
# and labels of that image belong to which class

Visualize some of the images from our dataset

In [ ]:
# printing 12 images 3*4 rows type with labels
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12,12+12):
        ax = plt.subplot(3, 4, i-12 + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

Train 80% Validation 10% Test 10%
- dataset is total batch unhi mein baaton

In [ ]:
len(dataset)

In [ ]:
print(68*0.8)
print(68*0.1)

In [ ]:
train_ds = dataset.take(54)
test_ds=dataset.skip(54) # after 54 rest

In [ ]:
validation_ds= test_ds.take(6)
test_ds=test_ds.skip(6) # rest inside test

Same Train Test Split using function

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    # aset makes sure condfition is true before going inside the funtion
    ds_size = len(ds)

    # shuffle the ds complety
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

Prefetch and Cache in Tf Dataset
- while cpu busy gpu concept
- load images before hand
- buffer_size=tf.data.AUTOTUNE let tf decide how many images to load beforehand

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Images -> Scaling /255 -> Data Augmentation sample images rotated zoomed

## Building the Model
- Sequential layers
- Layer1- resize and rescale
- Layer2 - data augmentation original imagte : rotate, contrast, zoom

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.Rescaling(1./255),
])

# why resizing we already did while extracting tf dataset
# if predict random image then it goes thorough this layer

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, RGB_CHANNELS)
n_classes = 3

# since we provide tf_dataset.take(0) --> 32 bathc image hai each
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()
# Paramsa are number of weights to be trained

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=1,
)
# recording in history of each epoch
# verbose=1 prints training and validation progress with a progress bar
# use this history to plot loss accuracy boom boom

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
scores

Plotting the Accuracy and Loss Curves

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
history.history['loss'][:5]

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(1), acc, label='Training Accuracy')
plt.plot(range(1), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(1), loss, label='Training Loss')
plt.plot(range(1), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Run prediction on a sample image

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print(batch_prediction)
    # complete batch prediction with each value probability of output 3 classes
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

Write a function for inference
- need input as batch since thats way we have layered model predict
- output as batch array

In [ ]:

#model is in batch form (32 image ouput 32 single araay of 3)
# WE want single image prediction single ouput [0,0,0] type

def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    # prediction is in batch type so index 0
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        # Plot inside subplot divide
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")

        plt.axis("off")

Saving the model of Keras after training

In [ ]:
model.save("/content/Export/my_model.keras")

In [39]:
MODEL = tf.keras.models.load_model("/content/Export/my_model.keras")

In [ ]:
MODEL.summary()

In [ ]:
model.save("/content/Potato/my_model.h5")
# for tf model

In [ ]:
model.export("/content/Potato/my_model/")

In [ ]:
model.save("/content/Potato/my_model/")

### Locally Download the model

In [ ]:
import shutil

shutil.make_archive('potatoModel', 'zip', '/content/Potato/my_model')
from google.colab import files

files.download('potatoModel.zip')

In [ ]:
from google.colab import files
files.download('/content/Export/my_model.keras')

# Tf Serving -> API Server using Docker HTTP Server
- Docker
# Deployment Cloud